In [1]:
# Copyright © 2019-2021 Intel Corporation.
# 
# This software and the related documents are Intel copyrighted
# materials, and your use of them is governed by the express 
# license under which they were provided to you (License). Unless
# the License provides otherwise, you may not use, modify, copy, 
# publish, distribute, disclose or transmit  this software or the
# related documents without Intel's prior written permission.
# 
# This software and the related documents are provided as is, with
# no express or implied warranties, other than those that are 
# expressly stated in the License.

In [2]:
import os
import nxsdk.api.n2a as nx
import time

from nxsdk_modules.slayer.src.slayer2loihi import Slayer2Loihi as s2l
from gestureDataset import IBMGestureDataset

os.environ['SLURM'] = '1' 
os.environ['PARTITION'] = 'nahuku32'

### Get the latest SLAYER models

In [3]:
modelPath = s2l.getModels() + '/03_IBMGesture/'

### Get the DVS Gesture dataset

In [4]:
dataset = IBMGestureDataset('/nfs/ncl/datasets/DVSgesture')

In [5]:
spikesPerPacket = 2048

# Load a previously compiled board. If you change the model, set this to False to regenerate the board
loadState = False
# Save the board after compilation so it can be loaded from file. Set to true if generating a new board
saveState = False

boardName = 'dvs_gesture'
regenerateCoreAxon = not loadState

# How many Lakemonts to distribute spike injection across
numSnips = 1

# The NeuroCore from which we'll start placing compartments
corenum = 0 

# create an empty network
net = nx.NxNet()

compProto = s2l.compartmentPrototype(modelPath+'network.yaml')

### Model
| Layer    |      Kernel    |  Output |
|----------|:-------------:|------:|
| input |  1a | 128x128x2 |
| 1 |  4a   | 32x32x2 |
| 2 |  16c5 | 32x32x16 |
| 3 |  2a | 16x16x16 |
| 4 |  32c3 | 16x16x32 |
| 5 |  2a | 8x8x64 |
| 6 |  10o | 11 |


### Input Layer
128 x 128 x 2

In [6]:
inputSpec = dict()

inputSpec["sizeX"] = 128
inputSpec["sizeY"] = 128
inputSpec["sizeC"] = 2
compartmentsPerCore = 1024

layerInput, inputConnectionGroup, corenum = s2l.inputLayer(net, inputSpec, corenum, compartmentsPerCore)

### Layer 1 
128 x 128 x 2 input <br>
4a pooling <br>
32 x 32 x 2 output <br>

In [7]:
poolSpec = dict()
poolSpec["stride"] = 4
poolSpec["compProto"] = compProto
poolSpec["weightFile"] = modelPath + 'Trained/pool1.npy'
compartmentsPerCore = 4096/16

layer1, corenum = s2l.poolingLayer(layerInput, poolSpec, corenum, compartmentsPerCore)

### Layer 2
32 x 32 x 2 input <br>
16c5z convolution <br>
32 x 32 x 16 output <br>

In [8]:
convSpec = dict()
convSpec["compProto"] = compProto
convSpec["dimX"] = 5
convSpec["dimY"] = 5
convSpec["dimC"] = 16
convSpec["weightFile"] = modelPath + 'Trained/conv1.npy'
compartmentsPerCore = 1024

layer2, corenum = s2l.convLayer(layer1, convSpec, corenum, compartmentsPerCore)

### Layer 3
32 x 32 x 16 input <br>
2a pooling <br>
16 x 16 x 16 output <br>

In [9]:
poolSpec = dict()
poolSpec["stride"] = 2
poolSpec["compProto"] = compProto
poolSpec["weightFile"] = modelPath + 'Trained/pool2.npy'
compartmentsPerCore = 1024

layer3, corenum = s2l.poolingLayer(layer2, poolSpec, corenum, compartmentsPerCore)

### Layer 4
16 x 16 x 16 input <br>
32c3z convolution <br>
16 x 16 x 32 output <br>

In [10]:
convSpec = dict()
convSpec["compProto"] = compProto
convSpec["dimX"] = 3
convSpec["dimY"] = 3
convSpec["dimC"] = 32
convSpec["weightFile"] = modelPath + 'Trained/conv2.npy'
compartmentsPerCore = 1024

layer4, corenum = s2l.convLayer(layer3, convSpec, corenum, compartmentsPerCore)

### Layer 5
16 x 16 x 32 input <br>
2a pooling <br>
8 x 8 x 32 output <br>

In [11]:
poolSpec = dict()
poolSpec["stride"] = 2
poolSpec["compProto"] = compProto
poolSpec["weightFile"] = modelPath + 'Trained/pool3.npy'
compartmentsPerCore = 256

layer5, corenum = s2l.poolingLayer(layer4, poolSpec, corenum, compartmentsPerCore)
layer5 = s2l.reorderLayer(layer5)

### Layer 6
8 x 8 x 32 input <br>
512 fully connected <br>
512 output <br>

In [12]:
fullSpec = dict()
fullSpec["compProto"] = compProto
fullSpec["dim"] = 512
fullSpec["weightFile"] = modelPath + 'Trained/fc1.npy'
compartmentsPerCore = 86

layer6, corenum  = s2l.fullLayer(layer5, fullSpec, corenum, compartmentsPerCore)

### Layer Output
512 input <br>
11 fully connected <br>
11 output <br>

In [13]:
fullSpec = dict()
fullSpec["compProto"] = compProto
fullSpec["dim"] = 11
fullSpec["weightFile"] = modelPath + 'Trained/fc2.npy'
compartmentsPerCore = 20

layerOutput, corenum = s2l.fullLayer(layer6, fullSpec, corenum, compartmentsPerCore)

In [14]:
dummyProbes = s2l.setupSpikeCounters(layerOutput)

In [15]:
if loadState is False:
    tStart = time.time()
    compiler = nx.N2Compiler()
    board = compiler.compile(net)
    tEnd = time.time()
    print("Completed compilation in {:.2f} seconds ".format(tEnd-tStart))
else:
    board, dummyProbes = s2l.initBoard(boardName)

Completed compilation in 272.01 seconds 


In [16]:
numSamples = len(dataset)

blankTime = 100 #how many blank timesteps gap between samples
spikeTime = dataset.sampleLength
sampleLength = spikeTime + blankTime

s2l.writeHeader(layerOutput, spikesPerPacket, sampleLength)
spikeChannels, core, axon = s2l.prepSpikeInjection(inputConnectionGroup, board, spikesPerPacket, sampleLength, numSnips, regenerateCoreAxon)
spikeData, numSteps = s2l.prepSpikeData(core, axon, spikesPerPacket, layerInput, dataset, numSamples, sampleLength, numSnips)

In [17]:
spikeCntrChannel = s2l.prepSpikeCounter(board, numSamples, layerOutput.numNodes, int(corenum))

In [18]:
board.start()
if saveState is True:
    s2l.saveBoard(board, boardName, dummyProbes)
if loadState is True:
    s2l.loadBoard(board, boardName)

INFO:DRV:  SLURM is being run in background
INFO:DRV:  Connecting to 134.134.68.92:33787
INFO:DRV:      Host server up..............Done 0.45s
INFO:DRV:      Encoding axons/synapses.....Done 6.74s
INFO:DRV:      Compiling Embedded snips....Done 0.77s
INFO:DRV:      Compiling MPDS Registers....Done 3.72ms
INFO:HST:  Config /etc/nx/pio.cfg
INFO:HST:  Args chip=0 cpu=0 /home/sshresth/nxsdk-nxsdk/nxsdk/driver/compilers/../../../temp/1604208662.5784721/launcher_chip0_lmt0.bin --chips=1 --remote-relay=0 --epoch=0 
INFO:HST:  Lakemont_driver...
INFO:DRV:      Booting up..................Done 7.92s
INFO:DRV:      Encoding probes.............Done 2.08ms


In [19]:
board.run(numSteps, aSync=True)

INFO:DRV:      Transferring probes.........Done 4.79ms
INFO:DRV:      Configuring registers.......Done 2.62s
INFO:DRV:      Transferring spikes.........Done 0.64ms


In [20]:
tStart = time.time()
s2l.sendSpikeData(spikeData, spikeChannels, spikesPerPacket)

In [21]:
numClasses = layerOutput.numNodes
results = s2l.getResults(spikeCntrChannel, numSamples, numClasses, dummyProbes)

In [22]:
board.finishRun()
board.disconnect()
tEnd = time.time()
print("Completed {} timesteps in {:.2f} seconds".format(numSteps, tEnd-tStart))

INFO:DRV:      Executing...................Done 1.92ms
INFO:DRV:      Processing timeseries.......Done 3.25ms
INFO:HST:  chip=0 cpu=0 halted, status=0x0
Completed 410200 timesteps in 70.26 seconds


In [23]:
labels = dataset.labels
accuracy = s2l.checkAccuracy(labels, results)
print('Final accuracy is {:.2f}%'.format(accuracy*100))

Final accuracy is 92.23%
